# Randomized Benchmarking experiment for tProc_v1

This notebook runs the experiment used in the QICK paper for randomized benchmarking (devForLBNL/RB_code_demo/qsystem2_paper_data-Ankur-final.ipynb) ported to run on the current tProc_v1 firmware (original code ran on something called qsystem, previous version of tProc_v1) It was created to measure the runtime of the algorithm and compare it with the implementation on the tProc_v2 firmware.

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
from datetime import datetime

# Import the QICK drivers and auxiliary libraries
from qick import *

# Load bitstream with custom overlay
soc = QickSoc()
soccfg = soc
print(soccfg)


QICK running on ZCU111, software version 0.2.370

Firmware configuration (built Wed Aug 16 13:39:03 2023):

	Global clocks (MHz): tProc dispatcher timing 384.000, RF reference 204.800
	Groups of related clocks: [tProc clock, DAC tile 0], [DAC tile 1], [ADC tile 0]

	7 signal generator channels:
	0:	axis_signal_gen_v6 - fs=6144.000 Msps, fabric=384.000 MHz
		envelope memory: 65536 complex samples (10.667 us)
		32-bit DDS, range=6144.000 MHz
		DAC tile 0, blk 0 is DAC228_T0_CH0, or RF board DAC port 0
	1:	axis_signal_gen_v6 - fs=6144.000 Msps, fabric=384.000 MHz
		envelope memory: 65536 complex samples (10.667 us)
		32-bit DDS, range=6144.000 MHz
		DAC tile 0, blk 1 is DAC228_T0_CH1, or RF board DAC port 1
	2:	axis_signal_gen_v6 - fs=6144.000 Msps, fabric=384.000 MHz
		envelope memory: 65536 complex samples (10.667 us)
		32-bit DDS, range=6144.000 MHz
		DAC tile 0, blk 2 is DAC228_T0_CH2, or RF board DAC port 2
	3:	axis_signal_gen_v6 - fs=6144.000 Msps, fabric=384.000 MHz
		envelope memo

## RB sequence generator

In [2]:
"""Writing the sequences in terms of Clifford group algebra
    The Clifford gate set forms a closed group => 
    1. Multiplication of any two gates results in a gate part of the group 
    2. Each gate has an inverse
    3. The inverse of a product of multiple Clifford gates is unique
"""
def generate_rbsequence(depth):
    
    """Single qubit RB program to generate a sequence of 'd' gates followed 
        by an inverse gate to bring the qubit back in 'g' state
    """

    gate_symbol = ['I', 'Z', 'X', 'Y', 'Z/2', 'X/2', 'Y/2', '-Z/2', '-X/2', '-Y/2']
    inverse_gate_symbol = ['I', '-Y/2', 'X/2', 'X', 'Y/2', '-X/2']

    """Modeled the bloch sphere as 6-node graph, each rotation in the RB sequence is effectively
    exchanging the node label on the bloch sphere.
    For example: Z rotation is doing this: (+Z->+Z, -Z->-Z, +X->+Y, +Y->-X, -X->-Y, -Y->+X)
    """
    matrix_ref = {}
    """Matrix columns are [Z, X, Y, -Z, -X, -Y]"""

    matrix_ref['0'] = np.matrix([[1, 0, 0, 0, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 1, 0, 0, 0],
                                 [0, 0, 0, 1, 0, 0],
                                 [0, 0, 0, 0, 1, 0],
                                 [0, 0, 0, 0, 0, 1]])
    matrix_ref['1'] = np.matrix([[1, 0, 0, 0, 0, 0],
                                 [0, 0, 0, 0, 1, 0],
                                 [0, 0, 0, 0, 0, 1],
                                 [0, 0, 0, 1, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 1, 0, 0, 0]])
    matrix_ref['2'] = np.matrix([[0, 0, 0, 1, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 0, 0, 0, 1],
                                 [1, 0, 0, 0, 0, 0],
                                 [0, 0, 0, 0, 1, 0],
                                 [0, 0, 1, 0, 0, 0]])
    matrix_ref['3'] = np.matrix([[0, 0, 0, 1, 0, 0],
                                 [0, 0, 0, 0, 1, 0],
                                 [0, 0, 1, 0, 0, 0],
                                 [1, 0, 0, 0, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 0, 0, 0, 1]])
    matrix_ref['4'] = np.matrix([[1, 0, 0, 0, 0, 0],
                                 [0, 0, 0, 0, 0, 1],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 0, 1, 0, 0],
                                 [0, 0, 1, 0, 0, 0],
                                 [0, 0, 0, 0, 1, 0]])
    matrix_ref['5'] = np.matrix([[0, 0, 1, 0, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 0, 1, 0, 0],
                                 [0, 0, 0, 0, 0, 1],
                                 [0, 0, 0, 0, 1, 0],
                                 [1, 0, 0, 0, 0, 0]])
    matrix_ref['6'] = np.matrix([[0, 0, 0, 0, 1, 0],
                                 [1, 0, 0, 0, 0, 0],
                                 [0, 0, 1, 0, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 0, 1, 0, 0],
                                 [0, 0, 0, 0, 0, 1]])
    matrix_ref['7'] = np.matrix([[1, 0, 0, 0, 0, 0],
                                 [0, 0, 1, 0, 0, 0],
                                 [0, 0, 0, 0, 1, 0],
                                 [0, 0, 0, 1, 0, 0],
                                 [0, 0, 0, 0, 0, 1],
                                 [0, 1, 0, 0, 0, 0]])
    matrix_ref['8'] = np.matrix([[0, 0, 0, 0, 0, 1],
                                 [0, 1, 0, 0, 0, 0],
                                 [1, 0, 0, 0, 0, 0],
                                 [0, 0, 1, 0, 0, 0],
                                 [0, 0, 0, 0, 1, 0],
                                 [0, 0, 0, 1, 0, 0]])
    matrix_ref['9'] = np.matrix([[0, 1, 0, 0, 0, 0],
                                 [0, 0, 0, 1, 0, 0],
                                 [0, 0, 1, 0, 0, 0],
                                 [0, 0, 0, 0, 1, 0],
                                 [1, 0, 0, 0, 0, 0],
                                 [0, 0, 0, 0, 0, 1]])
    
    """Generate a random gate sequence of a certain depth 'd'"""
    gate_seq = []
    for ii in range(depth):
        gate_seq.append(np.random.randint(0, 9))
    """Initial node"""
    a0 = np.matrix([[1], [0], [0], [0], [0], [0]])
    anow = a0
    for i in gate_seq:
        anow = np.dot(matrix_ref[str(i)], anow)
    anow1 = np.matrix.tolist(anow.T)[0]
    """Returns the """
    max_index = anow1.index(max(anow1))
    symbol_seq = [gate_symbol[i] for i in gate_seq ]
    symbol_seq.append(inverse_gate_symbol[max_index])
    return symbol_seq

In [3]:
generate_rbsequence(3)

['Z/2', 'Y', '-Z/2', 'X']

In [3]:
class RBSequenceProgram(AveragerProgram):
    def __init__(self,soccfg, cfg):
        super().__init__(soccfg, cfg)

    def initialize(self):
        cfg = self.cfg

        self.gate_seq = cfg['gate_seq']
        self.gate_set = cfg['gate_set']

#         r_freq=self.sreg(cfg["rr_ch"], "freq")   #Get frequency register for res_ch
#         f_res=freq2reg(adcfreq(cfg["rr_freq"]))  # convert frequency to dac frequency (ensuring it is an available adc frequency)

# #         rr_freq = self.sreg(self.device_cfg["rr_ch"], self.device_cfg["rr_freq"])   #Get frequency register for res_ch
#         self.cfg["adc_lengths"]=[self.cfg["rr_length"]]*2          #add length of adc acquisition to config
#         self.cfg["adc_freqs"]=[adcfreq(self.cfg["rr_freq"])]*2   #add frequency of adc ddc to config
#         self.f_ge=freq2reg(cfg["qubit_freq"])

        pulse_length = self.us2cycles(0.025)

        ro_chs = cfg['ro_chs']
        gen_ch = cfg['qubit_ch']

        # configure the readout lengths and downconversion frequencies
        for ro_ch in ro_chs:
            self.declare_readout(ch=ro_ch, 
                                 length=self.us2cycles(cfg['rr_length']),
                                 freq=self.cfg['rr_freq'],
                                 gen_ch=cfg['rr_ch'])

        self.declare_gen(ch=cfg["rr_ch"], nqz=1) #Readout

        """Add a sequence of fixed number of gates, analytically compute
            the inverse operation after the final gate to bring the qubit back
            to |g> state (minimizes errors due to readout) before reading out the state.
            Repeat this experiment several times with the fixed number of gates.
        """
#         self.add_pulse(ch=self.cfg["qubit_ch"], name="qubit", style="arb", idata=gauss(mu=cfg["pi_sigma"]*16*5/2,si=cfg["pi_sigma"]*16, length=5*cfg["pi_sigma"]*16, maxv=32000))

        
        """This adds the different types of pulses to the pulse library which can be played on demand later"""
        for name, ginfo in self.gate_set.items():
            self.add_pulse(ch=self.cfg["qubit_ch"], name=name, 
                           idata=ginfo["idata"],
                           qdata=ginfo["qdata"],
                        #    style=ginfo["style"]
                          )


        # self.add_pulse(ch=self.cfg["rr_ch"], name="measure",
        #             #    style="const", 
        #                length=self.cfg["rr_length"]
        #               )  #add a constant pulse to the pulse library of res_ch

        # """Pre-initialze the pulse (not necessary)"""
        # self.pulse(ch=self.cfg["rr_ch"], name="measure", freq=f_res, 
        #            phase=self.cfg["rr_phase"], gain=self.cfg["rr_gain"],
        #            length=self.cfg['rr_length'] , t=0, play=False) # pre-configure readout pulse

        idata = 30000*np.ones(16*cfg["rr_length"])
        self.add_pulse(ch=cfg['rr_ch'], name="measure", idata=idata)

        self.set_pulse_registers(ch=cfg["rr_ch"], 
                                 style="const", 
                                 freq=cfg["rr_freq"], 
                                 phase=0, 
                                 gain=cfg["rr_gain"],
                                 length=cfg["rr_length"])


        self.sync_all(1000)  # give processor some time to configure pulses
    
    def body(self):
        self.phase_ref=0
        for g in self.gate_seq:
            ginfo=self.cfg["gate_set"][g]
            """For the Z gates (virtual rotation), we need to advance the phase of all the pulses which follows afterwards"""
            if g=="Z":
                self.phase_ref+=180
            elif g=="Z/2":
                self.phase_ref+=90
            elif g=="-Z/2":
                self.phase_ref+=-90
            else:
                # self.pulse(ch=self.cfg["qubit_ch"], name=g, phase=deg2reg(self.phase_ref+ginfo["phase"]), gain=ginfo["gain"], play=True)
                pass
            self.setup_and_pulse(ch=self.cfg["qubit_ch"], 
                                    waveform=g, 
                                    phase=self.deg2reg(self.phase_ref+ginfo["phase"]), 
                                    gain=ginfo["gain"], 
                                    style=ginfo["style"],
                                    freq=self.cfg["qubit_freq"],
                                )

        self.sync_all(self.us2cycles(0.01)) # align channels and wait 10ns
        # self.trigger_adc(adc1=1, adc2=1, adc_trig_offset=self.cfg["adc_trig_offset"])  # trigger the adc acquisition
        self.trigger(adcs=self.cfg['ro_chs'],
                     pins=[0], 
                     adc_trig_offset=self.cfg["adc_trig_offset"])
        # self.pulse(ch=self.cfg["rr_ch"], length = self.cfg["rr_length"], play=True) # play readout pulse
        self.pulse(ch=self.cfg['rr_ch'], t=0) # play readout pulse
        self.sync_all(self.us2cycles(self.cfg["relax_delay"]))  # sync all channels

In [4]:
from qick.helpers import gauss

pi_sigma = 10    # us2cycles(0.025) -> tProc @384MHz -> 10 cycles
pi_gain = 1
pi_2_gain = 1
nu_q_new = 100
f_res = 100
rot_angle = 0

qubit_params = {'pi_sigma': pi_sigma, 'pi_gain': pi_gain, 'pi_2_gain': pi_2_gain}

RO_CH = [0,1]

rb_config = {
            "ro_chs"          : RO_CH, # --Fixed
            'qubit_freq'      : nu_q_new,   'qubit_ch'      : 3, 
            'rr_freq'         : f_res,      'rr_ch'         : 6, 
            'rr_length'       : 3, 
            'rr_gain'         : 10000,
            'rr_phase'        : rot_angle, 
            "adc_trig_offset" : 250,        'beta'          : 0,
            # 'relax_delay'   : 500,
            'relax_delay'     : 1,
            'reps'            : 1000,       'rounds'        : 0,
            'gate_seq'        : generate_rbsequence(3), 
            'gate_set'        : {
                        "I": {
                                "idata": gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000),
                                "qdata": 0*gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000),             
                                "phase":0, "gain":0, "style":"arb",
            
                                },
                        "X": {
                                "idata": gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000),
                                "qdata": 0*gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000), 
                                "phase":0, "gain":qubit_params['pi_gain'], "style":"arb",
                                },
                        "Y": {
                                "idata": gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                             length=4*pi_sigma*16,maxv=32000),
                                "qdata": 0*gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000),
                                "phase":-90, "gain":qubit_params['pi_gain'], "style":"arb",
                                },
                        "Z": {
                                "idata": gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000),
                                "qdata": 0*gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000), 
                                "phase":0, "gain":0, "style":"arb",
                                },
                        "X/2": {
                                "idata": gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000),
                                "qdata": 0*gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000), 
                                "phase":0, "gain":qubit_params['pi_2_gain'], "style":"arb",
                                },
                        "-X/2": {
                                "idata": gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000),
                                "qdata": 0*gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000), 
                                "phase":180, "gain":qubit_params['pi_2_gain'], "style":"arb",
                                },

                        "Y/2": {
                                "idata": gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000), 
                                "qdata": 0*gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000), 
                                "phase":-90, "gain":qubit_params['pi_2_gain'], "style":"arb",
                                },
                        "-Y/2": {
                                "idata": gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000), 
                                "qdata": 0*gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000), 
                                "phase":90, "gain":qubit_params['pi_2_gain'], "style":"arb",
                                },
                        "Z/2": {
                                "idata": gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000),
                                "qdata": 0*gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000), 
                                "phase":0, "gain":0, "style":"arb",
                                },
                        "-Z/2": {
                                "idata": gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000),
                                "qdata": 0*gauss(mu=pi_sigma*16*4/2,si=pi_sigma*16,
                                               length=4*pi_sigma*16,maxv=32000), 
                                "phase":0, "gain":0, "style":"arb",
                                },

                        } 
               }

# """Testing if it runs"""
# # config['gate_seq'] = generate_rbsequence(10)
# rb_config['gate_seq'] = ['Y', 'X/2']
# # rb_config['gate_seq'] = ['Y', 'X/2', 'X/2']
# rb_config['gate_seq'] = ['X', 'Y/2']


# print(rb_config['gate_seq'])
# rb_config['reps'] = 5000
# rb=RBSequenceProgram(soccfg, cfg=rb_config)
# results  = rb.acquire(soc, load_pulses=True, progress=True)
# np.mean(results[3])

## Experiment

In [5]:
# from datetime import datetime
import time

I2 = []
I2_err = []

# Original settings
depth_array = [1, 2, 3, 4, 5, 6, 8, 10, 12, 16, 20, 24, 32, 40, 48, 64, 80, 96, 128, 192, 
               256, 320, 384, 448, 512, 576, 640, 704, 768, 832, 896, 960, 1024, 1088, 1152, 1216, 1280, 1344, 1408,
               1472, 1536, 1600, 1664, 1728, 1750]
rb_config['reps'] = 4000
rb_config['rounds'] = 0
variety = 40

# # Test settings 1
# rb_config['reps'] = 4000
# depth_array = [1, 16, 128, 1024]
# variety = 2

# # Test settings 2
# rb_config['relax_delay'] = 100
# rb_config['reps'] = 4000
# depth_array = [1, 16, 64, 128, 512, 768, 1024]
# variety = 3

# # Test settings 3
# rb_config['relax_delay']    = 500    # us
# depth_array                 = [1, 1024]
# rb_config['reps']           = 4000
# variety                     = 5

# Test settings 4
rb_config['relax_delay']    = 500    # us
depth_array                 = [1, 16, 64, 128, 512, 768, 1024]
reps                        = 4000
variety                     = 40

exp_start_time_ns = time.time_ns()
print('- Experiment Start Time: %0d ns' % (exp_start_time_ns))

th = -30

# Loop for different lengths
for d in depth_array:
    sequence_time_ns = time.time_ns()
    print('  - Depth array %0d - time: %0d ns, elapsed: %0d ns' % (d,sequence_time_ns,(sequence_time_ns-exp_start_time_ns)) )
    i2_temp = []
    run_time_total_ns = 0
    # Loop for different random sequences
    print('    - Sequence run time: ', end=" ")
    for jj in range(variety):
        rb_config['gate_seq'] = generate_rbsequence(d)
        # print(rb_config['gate_seq'])
        rb = RBSequenceProgram(soccfg, cfg=rb_config)
        # avgi0, avgq0, avgi1, avgq1  = rb.acquire(soc, load_pulses=True, progress=False, debug=False, single_shot=True)
        run_time_start_ns = time.time_ns()
        adc1, adc2 = rb.acquire(soc, progress=False)
        run_time_end_ns = time.time_ns()
        run_time_total_ns += (run_time_end_ns-run_time_start_ns)
        print(' (%0d): %0d ms' % (jj, (run_time_end_ns-run_time_start_ns)/1e6), end=" ")
        avgi0 = adc1[0]
        avgq0 = adc1[1]
        avgi1 = adc2[0]
        avgq1 = adc2[1]
        df = pd.DataFrame(avgi1, columns=['data'])
        g = df['data'].apply(lambda x: 0 if x < th else 1).mean()
        i2_temp.append(g) 
    print('')
    I2.append(np.mean(i2_temp))
    I2_err.append(np.std(i2_temp)/np.sqrt(variety))
    seq_end_time_ns = time.time_ns()
    print('    - Sequence Total time: %0d ms (Run time: %0d ms - Overhead time: %0d ms)' % ((seq_end_time_ns - sequence_time_ns)/1e6, run_time_total_ns/1e6, (seq_end_time_ns - sequence_time_ns)/1e6 - run_time_total_ns/1e6))

exp_end_time_ns = int(time.time_ns())
print('- End Time: %0d - elapsed: %0d' % (exp_end_time_ns, (exp_end_time_ns-exp_start_time_ns)))


- Experiment Start Time: 1761678244290016903 ns
  - Depth array 1 - time: 1761678244293387583 ns, elapsed: 3370680 ns
    - Sequence run time:   (0): 2091 ms  (1): 2051 ms  (2): 2051 ms  (3): 2051 ms  (4): 2051 ms  (5): 2050 ms  (6): 2050 ms  (7): 2053 ms  (8): 2054 ms  (9): 2050 ms  (10): 2051 ms  (11): 2050 ms  (12): 2052 ms  (13): 2050 ms  (14): 2052 ms  (15): 2050 ms  (16): 2051 ms  (17): 2051 ms  (18): 2051 ms  (19): 2051 ms  (20): 2051 ms  (21): 2050 ms  (22): 2052 ms  (23): 2051 ms  (24): 2051 ms  (25): 2050 ms  (26): 2052 ms  (27): 2051 ms  (28): 2051 ms  (29): 2050 ms  (30): 2051 ms  (31): 2050 ms  (32): 2055 ms  (33): 2051 ms  (34): 2051 ms  (35): 2050 ms  (36): 2051 ms  (37): 2050 ms  (38): 2054 ms  (39): 2052 ms 
    - Sequence Total time: 82668 ms (Run time: 82109 ms - Overhead time: 559 ms)
  - Depth array 16 - time: 1761678326961805924 ns, elapsed: 82671789021 ns
    - Sequence run time:   (0): 2062 ms  (1): 2061 ms  (2): 2062 ms  (3): 2067 ms  (4): 2063 ms  (5): 2069 ms

In [ ]:
print(rb)